<a href="https://colab.research.google.com/github/ENV716/Energy_Modeling_F2022/blob/main/Lab/Lab08/Lab08_ImpementingLPs_Approach3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 08 - LPs in Pythin with Set(), Param() and Dictionarioes (Approach 3)**


## Initializing 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/')

Mounted at /content/drive


Installing Pyomo and glpk solver.

In [2]:
!pip install pyomo
!apt-get install -y -qq glpk-utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.7 MB 5.1 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123942 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Pr

Importing pyomo and solver.

In [3]:
from pyomo.environ import *
#Import solver
opt=SolverFactory('glpk')

## Compare differentt approaches





Recall: 
* Approach 1: With values/scalar and one constraint at a time (refer to Lab 06 file)
* Approach 2: With vector/matrices format, indexes and sets, adding constraints in blocks with for loops (refer to Lab 07)
* Approach 3: With Set(), Param(), dictionaries and functions (preferred)

## Problem formulation: manufacturing facility

We will implement the chemical solutions model again using model objects (Sets() and Param()), using dictionaries to specify parameters, indexing decision variables and constraints by sets, using user defined function to define constraints and objective function in a more general form.

Start by writing the LP formulation indexed by sets and in standard form. 

Sets: \\
$m∈ M:$ set of machines A and B \\
$p∈P:$ set of chemical solution types I and II

Parameters:  \\
$a_{m,p}:$ number of hours on machine m needed to produce chemical solution type $p$ \\
$H_m:$  number of hours available on machine $m$ \\
$C_p:$ profit gained from producing type $p$ \\

Decision Variables: \\
$x_p:$ number of units of type p to produce \\

Problem Formulation: \\  
$ max_x	\sum_{p\in P}c_p*x_p$ \\
$ s.t.	\sum_{p\in P}a_{m,p}*x_p ≤ H_m  \quad ∀ m \in M $ \\
$ \quad \quad \quad x_p ≥ 0 \quad \quad \quad \quad   ∀p\in P $ \\

Now let’s start by adding sets and parameters to the model. Because we are using objects Sets() and Param(), the assigned names should start with “model.”.

## Creating model, defining sets and parameters

In [8]:
#using sets and parameter
model=ConcreteModel()

#Sets
model.M=Set(initialize=['MA','MB'])  #set of machines
model.P=Set(initialize=['TypeI','TypeII']) #set of solution types

#Parameters
model.c=Param(model.P,initialize={'TypeI':800,'TypeII':600})  #profit for each unit Type
model.H=Param(model.M,initialize={'MA':60,'MB':48}) #hours available on each machine
model.a=Param(model.M,model.P,initialize={
    ('MA','TypeI'):4,
    ('MA','TypeII'):2,
    ('MB','TypeI'):2,
    ('MB','TypeII'):4}) #hours need for each unit at each machine

## Defining decision variables

Next define the decision variables, objective function and constraint also indexed by set. For generalization purpose note that we use user defined function to enter the expressions. Most of the formulation you will find on the internet will use this syntax. Don’t forget object “model” will always be an argument to your function. And if you need to add a constraint for all elements within a specific set, those elements should also be an argument to your function. Note that now instead if using the “exp=” we are using “rule=”.

In [9]:
#add dec variables
model.X=Var(model.P,domain=NonNegativeReals)

In [10]:
## Added for ilustration
model.X.pprint()

X : Size=2, Index=P
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
     TypeI :     0 :  None :  None : False :  True : NonNegativeReals
    TypeII :     0 :  None :  None : False :  True : NonNegativeReals


## Defining objective function

In [11]:
#add obj func by defining a function and then we call argument rule instead of expr
def obj_profit(model):
    return sum(model.c[p]*model.X[p] for p in model.P)
model.profit=Objective(sense=maximize,rule=obj_profit)

In [23]:
## Added for ilustration
model.profit.pprint()

profit : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : 800*X[TypeI] + 600*X[TypeII]


## Defining constraints

In [14]:
#similar idea for adding constraints
def mach_hours(model,m):
    return sum(model.a[m,p]*model.X[p] for p in model.P) <= model.H[m]
model.mach=Constraint(model.M,rule=mach_hours)

In [18]:
## Added for for ilustration
for m in model.M:
    print(model.mach[m].expr)

4*X[TypeI] + 2*X[TypeII]  <=  60
2*X[TypeI] + 4*X[TypeII]  <=  48


## Solving the model

Then all you need to do is solve the model and print the results.

In [17]:
#Solve model
opt.solve(model)

#Print results
print("Profit =",model.profit())
print("Decision Variables")
for p in model.P:
    print(model.X[p],model.X[p].value)

Profit = 13200.0
Decision Variables
X[TypeI] 12.0
X[TypeII] 6.0
